In [1]:
import twodlearn as tdl
import twodlearn.datasets.mnist
import twodlearn.datasets.cifar10
import tensorflow as tf
from functools import reduce

In [2]:
#dataset = tdl.datasets.mnist.MnistDataset(work_directory='data/mnist', reshape=False)
dataset = tdl.datasets.cifar10.Cifar10(work_directory='data/cifar10')
n_classes = dataset.train.y.shape[-1]
input_shape = dataset.train.x.shape[1:]

File already downloaded.


In [3]:
stacked = tdl.convnet.StackedLayers(name='classifier')
with tf.name_scope(stacked.scope):
    stacked.add(tdl.convnet.Conv2DLayer(kernel_size=[5,5], filters=32,
                                        bias={'trainable': False},
                                        input_shape=(None,)+input_shape))
    stacked.add(tf.keras.layers.ReLU())
    stacked.add(tf.keras.layers.MaxPool2D(pool_size=[2,2]))
    stacked.add(tdl.convnet.Conv2DLayer(kernel_size=[5,5], filters=64))
    stacked.add(tf.keras.layers.MaxPool2D(pool_size=[2,2]))
    stacked.add(tf.keras.layers.ReLU())
    conv_dims = stacked.compute_output_shape(input_shape=(None,)+input_shape)
    reduce_dims = reduce(lambda x, y: x*y, conv_dims.as_list()[1:])
    stacked.add(tf.keras.layers.Reshape((reduce_dims,)))
    stacked.add(tf.keras.layers.Dense(units=512))
    stacked.add(tf.keras.layers.ReLU())
    stacked.add(tf.keras.layers.Dense(units=n_classes))
    stacked.add(tf.keras.layers.Softmax())
    

#inputs = tf.keras.Input(shape=input_shape)
#predictions = stacked(inputs)
#model = tf.keras.Model(inputs=inputs, outputs=predictions)

In [4]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
tdl.core.variables_initializer(tdl.core.get_variables(stacked))\
   .run(session=tf.keras.backend.get_session())

In [8]:
model.fit(dataset.train.x, dataset.train.y, batch_size=32, epochs=5)

Epoch 1/5
45000/45000 [==============================] - 8s 173us/step - loss: 0.5680 - acc: 0.8012
Epoch 2/5
45000/45000 [==============================] - 8s 178us/step - loss: 0.4548 - acc: 0.8395
Epoch 3/5
45000/45000 [==============================] - 8s 172us/step - loss: 0.3593 - acc: 0.8729
Epoch 4/5
45000/45000 [==============================] - 8s 178us/step - loss: 0.2818 - acc: 0.9008
Epoch 5/5
45000/45000 [==============================] - 8s 169us/step - loss: 0.2183 - acc: 0.9230


In [7]:
model.evaluate(dataset.valid.x, dataset.valid.y)

5000/5000 [==============================] - 1s 103us/step


[0.9625186674118043, 0.6686]

In [5]:
print(tf.keras.backend.get_session().run(tf.report_uninitialized_variables()))

[]


In [5]:
stacked.layers[0].bias.trainable #.init()

False

In [8]:
stacked.layers[0].kernel

<tf.Variable 'classifier/Conv2DLayer/kernel/kernel:0' shape=(5, 5, 3, 32) dtype=float32>

In [8]:


stacked.layers[-2].build(input_shape=(None, 45))